
# `CV TEAM 과제` 3편 (신경망 학습) 
<img src="../resource/images/42LABS.png" width="200">

`제작` : `hocjeong(wjdghcks02@gmail.com)`

---

# `[⭐과제 진행시 유의사항⭐]`
- 본 과제는 코드로 구현하면서 학습하는 과제입니다.
- 본 과제는 `Google Colab`에서 실행하는 것을 권장합니다! 
- **해당 디렉토리 전체를 업로드 후 실행해주세요, 만약 주피터 노트북만 코랩에서 실행시 이미지가 깨져 보일 수 있습니다!!**
- 만약 `local`에서 실행시 필요한 라이브러리는 설치해 주세요!! 

- 해당 과제는 2개의 요소로 구성되어 있습니다.
    - 1. `[개념 설명 및 예시]`
    - 2. `[TODO]`
- 개념 설명이 경우 해당 부분을 읽으시면서 복습하시면 됩니다.
- 해당 개념에 대한 이해를 바탕으로 **`[TODO]`** 부분을 직접 작성하시면 됩니다.
- **과제를 수행하시다가 궁금한 사항이나 오류 발생시 `hocjeong`에게 문의 해주시면 됩니다!!**

---

# 목차
- [1. 데이터를 통한 학습](#1-데이터를-통한-학습-from-data)
- [2 손실함수](#2-손실함수-loss-function)
    - [2.1 오차제곱합](#21-오차-제곱합-sum-of-squares-for-error)
    - [2.2 교차 엔트로피 오차](#22-교차엔트로피-오차-cross-entropy-error)
    - [TODO 1. 교차 엔트로피 오차 구현하기](#todo-1-교차-엔트로피-오차-구현하기)

우리는 지난시간까지 `신경망이 무엇인지`, `신경망는 어떤 요소들로 구성되어 있는지`, 그리고 `신경망의 순전파의 과정 및 구현`을 살펴보았습니다.  

이번 시간에는 이렇게 구성된 신경망을 `어떻게 학습할지`에 대해서 공부해봅시다.  

지난시간에 `신경망`은 가중치 업데이트를 자동으로 할 수 있다고 이야기했습니다.  
이번에 배울 내용의 핵심은 `어떻게 가중치를 자동으로 업데이트 할까`에 대한 원리입니다.  

# 1. 데이터를 통한 학습 (from data)
신경망의 가장 큰 특징은 `데이터`를 보고 학습을 한다는 것입니다.  
이는 머신러닝 전체에 관통하는 개념이지만, 약간의 차이가 존재합니다.  

예를 들어서 아래와 같은 숫자 데이터(MNIST)가 있다고 생각해봅시다.  
<img src="../resource/images/03/MNIST.png" width="400">  

인간의 경우 해당 이미지에서 `5`라는 숫자를 판단하기 위해서 어떤 과정을 거칠까요?  
정확하게 알 수 는 없지만 `뇌에서` 각 이미지를 보고 이미지의 특징을 판단해서 각 숫자로 맵핑을 할 것입니다..  

이를 프로그램으로 자동화 할 수 있을까요?   

`접근1. Rule 기반의 접근 방법?`  
가장 기초적인 프로그래밍 방법인 Rule 기반의 접근 방식으로 해당 이미지를 보고 숫자를 판단하는 프로그램을 작성하는 방법이다.    
각 숫자마다 어떤 pixel의 값에 따라서 분류하는 단순한 방법을 생각할 수 있습니다.    

하지만 해당 방법은 매우 한계가 있습니다.  
왜냐하면 사람마다 숫자를 쓰는 방법이 다르기 때문입니다. 같은 `5`라도 각 이미지 마다 특성이 조금씩 차이가 있기 때문입니다.  
또한 이를 `if else` 문으로 표현하려면 엄청나게 많은 조건문이 필요할 것입니다다.  
따라서 사실상 해당 방법은 불가능 합니다다. 

`접근2. 전통적인 기계학습? (SVM, KNN 등)`  
또다른 방법은 전통적인 기계학습 방법을 활용하는 것입니다.  
하지만 이러한 방법을 위해서는 `사람이 중요하게 생각하는 특징으로 변환해주는 알고리즘`을 먼저 사용해야합니다.  
따라서 `SIFT, HOG` 등의 방법으로 이미지를 변환하고 이를 기계학습에 input으로 넣어줘야 합니다.  
하지만 사람이 설계한 `SIFT, HOG` 등의 방법은 미쳐 고려지 못한 중요한 특징을 무시하게 되는 결과를 낳을 수 있습니다.  

`접근3. 신경망을 통한 방법`
우리가 배우고 있는 신경망을 활용하는 방법입니다.  
이는 기존 전통적인 기계학습 방법과 다르게 `데이터를 있는 그대로`학습합니다.  
즉, 사람의 주관 개입이 비교적 적거나 없다고 할 수 있습니다.  
이렇듯 처음부터 끝까지 사람의 개입없이 입력에서 출력을 얻는 것을 `종단간 기계학습 End-to-end machine learning`이라고하고  
딥러닝(신경망을 활용한)의 다른 이름입니다.  

물론 `신경망도` 일반적인 머신러닝 모델처럼 `학습 데이터 train data`와 `검증 데이터 또는 시험데이터 (validation data, test data)`로 구성해서 학습을 하고 모델의 과적합을 평가해야합니다.  

# 2. 손실함수 (loss function)
그렇다면 모델은 어떠한 기준으로 학습이 되어질까요?  
다른 말로 어떻게 매개변수를 자동으로 조정하여 데이터에서 중요한 특징을 추출하는 방법을 알게 할 수 있을까요?  
이러한 의문에 대답을 해주는 것이 `손실함수 loss function`입니다.  
손실함수는 `신경망 학습`에서 `지표`역할을 합니다.   

비유로 표현하자면 `대한민국 사람들은 얼마나 행복한가?`라는 질문을 `아주 많이 행복하다`라고 대답을 하면 먼가 기준이 애매해 보입니다.  
이를 `행복지수` 같이 수치적인 값으로 변환해서 `10.2 행복지수`만큼 행복합니다. 라고 대답을 한다면  
조금더 명확할 것입니다.  

손실함수가 신경망 학습에서 이러한 역할을 합니다.  
일반적으로 신경망에서는 손실함수를 최소로 만드는 방향으로 가중치를 업데이트해 나갑니다.  

## 2.1 오차 제곱합 sum of squares for error
가장 많이 사용되는 손실함수는 `오차 제곱합`입니다.  
수식으로 표현하면 다음과 같습니다.  
$$E=\frac{1}{2} \sum_{k} ({y_k} - {t_k})^2$$


여기서  
$y_k$는 신경망의 출력(신경망이 예측한 값)  
$t_k$는 정답 레이블  
$k$는 데이터의 차원수를 나타냅니다.

예를 들어서 `MINST`데이터 에서 하나의 이미지를 신경망을 통과해서 나온 값이 y이고  
해당 이미지의 label이 t라면 아래와 같이 표현할 수 있습니다.  
```python
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
```
여기서 y는 각각 라벨이 있때의 값이고(확률)  
t는 정답 라벨을 원핫 인코딩의 표현으로 나타낸 것입니다.  

이를 이제 위에 공식에 넣어서 `오차제곱합`을 구할 수 있습니다.   
`오차 제곱합`을 코드로 구현하면 다음과 같습니다. 


In [1]:
# 오차 제곱합
import numpy as np

def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t) ** 2)

앞선 예시를 한번 넣어볼까요?  

In [ ]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

sum_squares_error(np.array(y), np.array(t))

0.09750000000000003

정답이 2번이기 이고 이를 0.으로 가장 높게 예측했기 때문에 낮은 오차제곱합이 나온 것을 확인할 수 있습니다.  
이번에는 정답의 예측을 빗나가게 해볼까요?  

In [4]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

sum_squares_error(np.array(y), np.array(t))

0.5975

정답은 그대로지만 `7`을 가장 높은 확률로 예측하게 변경했습니다.  
오차제곱합의 변화가 눈에 들어오시나요?  

즉, 오차제곱합은 예측이 실제에 가까울 수록 값이 줄어들고 멀수록 값이 커집니다.  

## 2.2 교차엔트로피 오차 cross entropy error
교차 엔트로피 오차도 많이 사용되는 오차중 하나입니다.  
수식은 다음과 같습니다.  

$$E=- \sum_{k} t_k \log {y}_{k}$$

여기서
$log$는 밑이 $e$인 자연로그 입니다.  
오차 제곱합과 똑같이  
$y_k$는 신경망의 출력(신경망이 예측한 값)  
$t_k$는 정답 레이블  
입니다. 

따라서 실질적으로 정답일때의 추정의 자연로그를 계산하는 식이됩니다.  

예를 들어서 정답이 `2`일때, 이때 신경망의 출력이 0.6이하라면 교차 엔트로피 오차는 `-log0.6 = 0.51`이 됩니다.  
또한 같은 조건에서 0.1이라면 -> `-log0.1 = 2.30`이 됩니다.  

마찬가지로 정답에 해당하는 출력이 커질 수록 0에 다가가다가, 그 출력이 1일때 0이 됩니다.  
반대로 정답일때의 출력이 작아질수록 오차는 커집니다.  

이제 교차엔트로피 오차를 직접 구현해봅시다.
## TODO 1. 교차 엔트로피 오차 구현하기
[힌트]  
- 아주 작은 값이 되는 것을 방지해야합니다.  

In [5]:
# TODO 1. 교차엔트로피 오차 구현
# 아래 함수를 완성해 주세요

def cross_entropy_error(y, t):
    pass

제대로 구현하셨나요?  
아래 테스트 코드로 확인을 해봅시다.

In [6]:
from test_code import TEST_CROSS_ENTROPY

TEST_CROSS_ENTROPY(cross_entropy_error)

TEST result 0 / 5 Pass


## 2.3 미니배치 학습
기계학습 문제는 훈련 데이터를 사용해서 모델을 학습합니다.  
만약 훈련데이터가 100개가 있다면 100개의 손실함수의 값들의 합을 지표로 삼는 것입니다.  

위에서는 하나에 대한 손실 함수만을 생각했기 때문에  
이제는 모든 데이터에 대한 손실 함수의 합을 구하는 방법에 대해서 살펴봅시다.  

사실 간단합니다. 데이터 수 `N` 만큼 합하고 이를 평균내면 됩니다.  
예를 들어서 교차 엔트로피 오차를 위를 적용한 수식으로 표현하면 다음과 같습니다.  

$$E=-\frac{1}{N}\sum_{n}^{} \sum_{k}^{} t_{nk} \log {y}_{nk}$$

즉 `평균 손실함수`로 변경을 해주는 것입니다.  
이렇게 평균을 구해서 손실함수를 구하면 통일된 지표를 언제든지 얻을 수 있습니다.  

하지만 신경망을 학습할때는 모든 데이터를 한번에 학습에 투입하지 않습니다.  
하드웨어 적인 한계가 있기 때문에 `미니배치 단위`로 나눠서 투입을 합니다.  
즉, 배치 단위로 생각을 한다면 `미니배치 평균 손실함수`는 `근사치`로 볼 수 있습니다. 

한 가지 TIP이 있습니다. 신경망 또는 딥러닝은 `미니배치`로 가중치를 업데이트하기 때문에  
미니배치를 분포하는 데이터의 영향을 받을 수 있습니다.   
물론 큰수의 법칙으로 전체 데이터셋의 분포를 따라가겠습니다.   
데이터셋이 불균형한 경우 문제가 생길 수 있습니다.   
이를 해결하기 위해서 배치에 들어가는 클래스 분포를 맞춰주는 것도 한가지 팁입니다 

이제 `교차 엔트로피 오차`에 미니배치를 적용해 봅시다.

In [7]:
# 교차 엔트로피 미니배치 오차
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e7)) / batch_size

만약 정답레이블을 `원핫인코딩이 아니라` `7`, `2`와 같이 label의 단일한 숫자가 주어졌을때  
구하는 교차 엔트로피 미니배치 오차도 구할 수 있습니다. 

In [9]:
# 교차 엔트로피 미니배치 오차
def cross_entropy_error(y, t):
    if y.ndim == 1:  # 데이터 하나당 교차 엔트로피 오차를 구하는 경우 shape 변경
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

그렇다면 왜 손실 함수를 설정할까요?

왜 굳이 굳이 사용해야할까요? `정확도` 라는 지표도 사용할 수 있지 않을까요?  
이는 뒤에 나오는 `미분` 부분과 연결됩니다.  
손실함수를 이용해서 `미분`을 통해서 `기울기`를 구하고 이 `기울기`를 활용해서 `경사하강법`으로 가중치를 업데이트합니다.  

즉, 정확도(계단형 함수) 같이 범주형의 값은 미분이 불가능하기 때문에 사용할 수 없습니다.  
즉, 손실함수를 사용하는 이유는 `미분을`통해서 가중치를 업데이트 하기 위해서 입니다.  

여기서 `손실함수의 미분`은 가중치를 조금 변경했을때 `손실함수의 값`이 얼마나 변화하는 지를 의미합니다  

# 3. 수치 미분